In [1]:
import os
import sys
import csv
import nltk
import pandas as pd
import re
import gensim
import tensorflow as tf
import count_data as cd
import load_data as ld
import sklearn
import random

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from gensim import corpora
from gensim.test.utils import get_tmpfile, common_texts
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

TaggedDocument = gensim.models.doc2vec.TaggedDocument

In [ ]:
# APP350数据导出

input_dir = r'C:\python_project\ML\original_documents'
get_filename = tf.io.gfile.listdir(input_dir)  # 得到csv文件名列表
filenames = []
path_format = os.path.join(input_dir, '{}')
for filename in get_filename:
    part_csv = path_format.format(filename)
    filenames.append(part_csv)
    
# segments[]存储APP350文本

segments = []
for filename in filenames:
    file = open(filename, encoding='ISO-8859-1', errors='ignore')
    soup = BeautifulSoup(file, 'html.parser')
    segments.append(soup.get_text())

In [2]:
(x_train, y_train), (x_test, y_test) = ld.load_data('data/clean_data_0.5_no_repeat.csv')

In [8]:
print(str(y_train[2].numpy()))

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]


In [9]:
datasets = []
for data in x_train:
    datasets.append(data)
#for data in x_test:
#    datasets.append(data)
#for data in segments:
#    datasets.append(data)

In [10]:
#  利用tokenizer进行分词去标点等操作
tokenizer = Tokenizer(oov_token='<OOV>', lower=True)
tokenizer.fit_on_texts(datasets)
word_index = tokenizer.word_index
datasets = tokenizer.texts_to_sequences(datasets)

In [11]:
datasets = tokenizer.sequences_to_texts(datasets)
print(datasets[0])

privacy policy sci news com be committed protect respect your privacy well inform you our policy concern user privacy we have adopt following term please note that these term be subject change any such change will be include on this page


In [12]:
datasets = [segment.split() for segment in datasets]

In [13]:
def labelizeReviews(reviews):
    labelized = []
    for i,v in enumerate(reviews):
        labelized.append(TaggedDocument(words=v, tags=['SENT_%s' % i, str(y_train[i].numpy())]))
    return labelized

datasets = labelizeReviews(datasets)

In [15]:
model= Doc2Vec(min_count=1, window=5, vector_size=100, sample=1e-3, negative=5, workers=3,dm=0) 
ifmodel_vo=model.build_vocab(datasets)
for epoch in range(20): 
    random.shuffle(datasets)
    model.train(datasets,total_examples=len(datasets),epochs=model.epochs)
#    model.alpha -= 0.002
#    model.min_alpha = model.alpha

In [16]:
model.save('data/doc2vec+label.d2v')